In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import pdb

In [2]:
alpha_list = [  'eps','pb','pe','roa','dividend',
                'Alpha1','Alpha4','Alpha5','Alpha13','Alpha15','Alpha16','Alpha18',
                'Alpha25','Alpha26','Alpha33','Alpha35','Alpha37','Alpha57','Alpha84',
                'Alpha101','Alpha103']

alpha_idx = ['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10',
             'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19',
             'x20', 'x21']


In [3]:
data=pd.read_csv("./data/csi300_alpha_data/merge_data.csv")

data.rename(columns={'return'       :'y',
                     alpha_list[0]  :alpha_idx[0],
                     alpha_list[1]  :alpha_idx[1],
                     alpha_list[2]  :alpha_idx[2],
                     alpha_list[3]  :alpha_idx[3],
                     alpha_list[4]  :alpha_idx[4],
                     alpha_list[5]  :alpha_idx[5],
                     alpha_list[6]  :alpha_idx[6],
                     alpha_list[7]  :alpha_idx[7],
                     alpha_list[8]  :alpha_idx[8],
                     alpha_list[9]  :alpha_idx[9],
                     alpha_list[10] :alpha_idx[10],
                     alpha_list[11] :alpha_idx[11],
                     alpha_list[12] :alpha_idx[12],
                     alpha_list[13] :alpha_idx[13],
                     alpha_list[14] :alpha_idx[14],
                     alpha_list[15] :alpha_idx[15],
                     alpha_list[16] :alpha_idx[16],
                     alpha_list[17] :alpha_idx[17],
                     alpha_list[18] :alpha_idx[18],
                     alpha_list[19] :alpha_idx[19],
                     alpha_list[20] :alpha_idx[20],
                     },inplace=True)

In [4]:
x=data.copy()
del x['datetime']
del x['tickers']
del x['y']
y = data['y']

tmp = pd.concat([data['datetime'],data['tickers'],data['y']]   ,axis=1)
tmp=tmp.pivot(index='datetime', columns='tickers', values='y')
tickers_num = len(tmp.columns)
date_num = len(tmp)

print "ticker number: ", tickers_num
print "date number: ", date_num

print "return matrix: "
y
print "alpha matrix: "
x.head()

ticker number:  581
date number:  2958
return matrix: 
alpha matrix: 


,x1,x4,x2,x3,x5,x6,x7,x8,x9,x10,...,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21
0,0.528177,0.009611,13.130871,3.258554,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,1558.0,NaN,1635.0,NaN,NaN,NaN,NaN,-0.896803,0.400859
1,0.528177,0.009611,13.130871,3.258554,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,1569.0,NaN,1556.0,NaN,NaN,NaN,NaN,-0.390373,0.236017
2,0.528177,0.009611,13.130871,3.258554,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,1466.0,NaN,1312.0,NaN,NaN,NaN,NaN,-0.656344,-0.271192
3,0.528177,0.009611,13.130871,3.258554,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,1099.0,NaN,515.0,NaN,NaN,NaN,NaN,-0.383003,-0.388376
4,0.528177,0.009611,13.130871,3.258554,0.0,NaN,NaN,NaN,-307.0,-2855.0,...,NaN,1397.0,NaN,1168.0,NaN,NaN,NaN,NaN,0.248854,0.389951


In [8]:
date_range = 60
data_step = 20

r2=pd.DataFrame()
model = LinearRegression()
result=pd.DataFrame(data=[],columns=['index', 'score'])
result.to_csv('result.log',mode='w')
for i in range (0, date_num-date_range):

    x_train = pd.DataFrame()
    y_train = pd.DataFrame()
    x_test = pd.DataFrame()
    y_test = pd.DataFrame()
    print "Doing regression: ",i
    for j in range(0, tickers_num):
        x_train = pd.concat([x_train, x[j*date_num+i*data_step:date_range+j*date_num+i*data_step]])
        y_train = pd.concat([y_train, y[j*date_num+i*data_step:date_range+j*date_num+i*data_step]])
        x_test  = pd.concat([x_test,  x[date_range+j*date_num+i*data_step:date_range+j*date_num+i*data_step+1]])
        y_test  = pd.concat([y_test,  y[date_range+j*date_num+i*data_step:date_range+j*date_num+i*data_step+1]])

        # fill nan with mean value
        for idx in alpha_idx:
            x_train[idx].fillna( (x_train[idx].sum())/(len(x_train[idx])), inplace=True)
            x_test[idx].fillna( (x_train[idx].sum())/(len(x_train[idx])), inplace=True)

        y_test.fillna( (y_train.sum())/(len(y_train)), inplace=True)
        y_train.fillna( (y_train.sum())/(len(y_train)), inplace=True)

    model.fit(x_train,y_train)
    prediction = model.predict(x_test)
    score=model.score(x_test, y_test)
    r2=r2.append([score])

    result=result.append(pd.DataFrame({'index':[i],
                                       'score': score }))
    result=result[['index', 'score']]
    result[i:i+1].to_csv('result.log',mode='a',header=None,index=False)

    print "R-squared: %.2f" % score
    print "coef is :\n" , model.coef_
    print "Expected return: \n", y_test
    print "Predicted return: \n", prediction
    print "\n\n\n"


Doing regression:  0
R-squared: 0.71
coef is :
[[ 5.07341601e-04 -1.50892436e-02  1.34812756e-06  1.77721335e-04
  -8.98888008e-03  1.97686104e-07 -5.44550370e-05 -5.93559454e-10
  -2.82872130e-06  2.06288151e-07 -4.85282964e-07 -6.48805231e-07
  -8.58521053e-06 -6.76776745e-04  8.75975674e-08  5.72486312e-07
   9.76195395e-07 -1.12578657e+00 -1.18593830e-06  3.21177853e-02
  -1.97080320e-03]]
Expected return: 
                0
60       0.007355
3018    -0.003191
5976    -0.006974
8934    -0.010272
11892    0.008135
14850    0.003712
17808   -0.004488
20766   -0.001627
23724   -0.007336
26682   -0.002056
29640   -0.004832
32598    0.000000
35556    0.020991
38514   -0.017219
41472    0.001104
44430    0.000694
47388   -0.001244
50346    0.001395
53304    0.035787
56262   -0.008264
59220    0.009328
62178    0.011391
65136   -0.025641
68094   -0.001488
71052    0.009672
74010    0.002441
76968   -0.003810
79926    0.005601
82884    0.016860
85842   -0.015642
...           ...
1629918  

R-squared: 0.75
coef is :
[[ 5.88838958e-04 -1.48594303e-02  1.56328459e-06  1.78137516e-04
  -1.01418857e-02  3.83536276e-07 -6.09952905e-05 -8.64128326e-10
  -2.31924142e-06  3.34915950e-07 -6.19087221e-07 -1.02921678e-06
  -7.90033386e-06 -6.27286646e-04 -1.14066041e-06  6.62657037e-07
   8.94986898e-07 -1.67574231e+00 -1.55747036e-06  2.84643261e-02
  -1.11415952e-03]]
Expected return: 
                0
80      -0.021939
3038    -0.017023
5996    -0.006974
8954    -0.007145
11912   -0.034934
14870   -0.025537
17828   -0.031075
20786   -0.024411
23744   -0.029398
26702    0.006330
29660   -0.024044
32618   -0.027362
35576   -0.024865
38534   -0.016721
41492    0.004209
44450   -0.038006
47408   -0.029436
50366   -0.037604
53324    0.035787
56282   -0.020128
59240    0.009328
62198   -0.027387
65156   -0.025641
68114   -0.022088
71072    0.009672
74030    0.002342
76988   -0.024038
79946    0.005601
82904    0.016860
85862    0.037917
...           ...
1629938  0.009573
1632896  0.0

KeyboardInterrupt: 